In [1]:
import openpyxl

In [2]:
%run MyFunctions.ipynb

not suicide


In [3]:
## Import data
#filename = "ResearchInChildAndAd_DATA_2018-12-14_1531.xlsx"
filename = "FULLWITHOUTTYLERSSTUFFResearchInChildAndAd_DATA_2018-12-14_1531.xlsx"
workbook = openpyxl.load_workbook(filename)
sheet1 = workbook["ResearchInChildAndAd_DATA_2018-"]
#sheet2 = workbook["Tasks"]
print(workbook.sheetnames)
sheet3 = workbook["Illness Keywords"]
sheet4 = workbook["MRR Keywords"]
sheet5 = workbook["Symptoms"]

['ResearchInChildAndAd_DATA_2018-', 'Illness Keywords', 'MRR Keywords', 'Symptoms']


In [4]:
## Make columns for new data
IGNORE_KEYWORDS = ['query', 'vs', 'r/o', 'rule out', 'versus']

## Get illnesses
sheet3_columns = get_column_names(sheet3)
ILLNESS_KEYWORDS = read_in_illness_keywords(sheet3, sheet3_columns)
ILLNESSES = get_key_list(ILLNESS_KEYWORDS)

## Get reasons for referral
sheet4_columns = get_column_names(sheet4)
REASONS_FOR_REFERRAL_KEYWORDS = read_in_reasons_for_referral(sheet4, sheet4_columns)
REASONS_FOR_REFERRAL = get_key_list(REASONS_FOR_REFERRAL_KEYWORDS)

## Get other symptoms
sheet5_columns = get_column_names(sheet5)
print(sheet5_columns)
SYMPTOMS_KEYWORDS = read_in_symptoms_keywords(sheet5, sheet5_columns)
SYMPTOMS = get_key_list(SYMPTOMS_KEYWORDS)
print(SYMPTOMS)
print("**********")
print(SYMPTOMS_KEYWORDS)

{'Symptoms': 0, None: 3}
['NSSI', 'Suicidal']
**********
{'NSSI': ['nssi', 'self', 'cutt'], 'Suicidal': ['suicid', 'si']}


In [5]:
print(ILLNESSES)
print("*******************")
print(ILLNESS_KEYWORDS)
print("*******************")
print(REASONS_FOR_REFERRAL)
print("*******************")
print(REASONS_FOR_REFERRAL_KEYWORDS)
print("*******************")


['Neurodevelopmental', 'ADHD', 'Intellectual Disability/Global Developmental Delay', 'ASD', 'FASD', 'Schizophrenia Spectrum and Other Psychotic Disorders', 'Bipolar', 'Depression', 'Anxiety', 'OCD', 'Trauma', 'Dissociative', 'Somatic', 'Eating', 'Elimination', 'Sleep-Wake Disorders', 'Gender Dysphoria', 'Disruptive, Impulse-Control, and Conduct', 'Substance-Related and Addictive', 'Personality', 'Parent-Child Relational Issues', 'Learning']
*******************
{'Neurodevelopmental': ['neurodevelopment', 'cerebral', 'palsy', 'chromosom', 'xxx', 'triplex', 'tic'], 'ADHD': ['attention', 'adhd', 'add'], 'Intellectual Disability/Global Developmental Delay': ['intellectual', 'id', 'global', 'gdd', 'delay'], 'ASD': ['autis', 'asd'], 'FASD': ['fasd', 'fetal'], 'Schizophrenia Spectrum and Other Psychotic Disorders': ['schizo', 'psychos', 'psychot', 'delusion', 'catatoni'], 'Bipolar': ['bipolar', 'mani'], 'Depression': ['depress', 'mdd', 'pdd', 'mood', 'dysthymi', 'premenstrual', 'seasonal'], 'A

In [6]:
## Add columns
set_headers(sheet1, 'admission', ILLNESSES)
set_headers(sheet1, 'discharge', ILLNESSES)

set_referral_reason_headers(sheet1, REASONS_FOR_REFERRAL)
set_symptom_headers(sheet1, SYMPTOMS)

In [7]:
## Create a dictionary of column names
column_names = get_column_names(sheet1)

In [8]:
print(column_names)

{'record_id': 0, 'patient_no': 1, 'no_of_admissions': 2, 'event_no': 3, 'other_admission_IDs': 4, 'date_of_birth': 5, 'cape_admit_date': 6, 'date_discharge': 7, 'age_months_cape_admit_date': 8, 'age_years_cape_admit_date': 9, 'length_stay': 10, 'sex': 11, 'gender_id': 12, 'ethnicity___white': 13, 'ethnicity___southasian': 14, 'ethnicity___eastasian': 15, 'ethnicity___black': 16, 'ethnicity___filipino': 17, 'ethnicity___hispanic': 18, 'ethnicity___aboriginal': 19, 'ethnicity___other': 20, 'ethnicity__specify': 21, 'non_canadian': 22, 'country_of_patients_birth': 23, 'patients_first_native_lang': 24, 'parent_immigration': 25, 'prev_assessments': 26, 'prev_counselling': 27, 'prev_psych_med': 28, 'prev_psych_hosp___CAPE': 29, 'prev_psych_hosp___P1': 30, 'prev_psych_hosp___P2': 31, 'prev_psych_hosp___P3': 32, 'prev_psych_hosp___other': 33, 'prev_psych_hosp': 34, 'prev_mental_health_care': 35, 'prev_suicide': 36, 'suicide_attempt': 37, 'suicide_attempt_when': 38, 'aggression_hx': 39, 'hx_sub

In [9]:
## Module 1: Set binary values to 0
set_binaries_to_zero(sheet1, "admission", ILLNESSES)
set_binaries_to_zero(sheet1, "discharge", ILLNESSES)

set_mrr_to_zero(sheet1, REASONS_FOR_REFERRAL)
set_symptoms_to_zero(sheet1, SYMPTOMS)

In [10]:
## Module 2: Fill in main diagnosis
set_main_diagnosis(sheet1, "admission", IGNORE_KEYWORDS, ILLNESS_KEYWORDS)
set_main_diagnosis(sheet1, "discharge", IGNORE_KEYWORDS, ILLNESS_KEYWORDS)

*******************
['disinhibited social engagement disorder', 'unspecified trauma and stressor related disorder', 'adhd', 'combined type', 'fasd', 'mild intellectual disability', '']
*******************
['unspecified depressive disorder (r/o mdd)', 'unspecified anxiety disorder (themes of social phobia', 'panic disorder)', 'parent-child relational problem', 'somatic symptom disorder with pain', '']
*******************
['query unspecified psychosis versus bipolar disorder', 'query asd', 'query adhd', '']
*******************
['situational crisis', 'adjustment disorder', 'r/o depressive disorder', '']
*******************
['situational crisis', 'parent-child relational problem', 'unspecified depressive disorder.']
*******************
['acute suicidal ideation with recent multi-drug intentional overdose with intent to die', 'unspecified anxiety disorder', 'significant psychosocial stressors related to lack of consistent primary attachments and frequent multiple housing and environmental c

*******************
['n/a']
*******************
['n/a']
*******************
['n/a']
*******************
['n/a']
*******************
['n/a']
*******************
['n/a']
*******************
['broderline personality traits versus disorder', 'unspecified depressive disorder', 'unspecified anxiety disorder', 'cannabis use disorder', 'mild', 'parent-child relational problem', 'intellectual disability (learning disorder in written output)', '']
*******************
['acute stress reaction', 'major depressive disorder', 'adhd', '']
*******************
['n/a']


In [11]:
## Module 3: Read Diagnosis, break into substrings, fill in binaries
one_hot_encode_diagnoses(sheet1, 'admission', IGNORE_KEYWORDS, ILLNESS_KEYWORDS)
one_hot_encode_diagnoses(sheet1, 'discharge', IGNORE_KEYWORDS, ILLNESS_KEYWORDS)

*****************
['Trauma', 'Disruptive, Impulse-Control, and Conduct']
adjustment disorder with disturbance of conduct
*****************
['Schizophrenia Spectrum and Other Psychotic Disorders', 'Depression']
major depressive disorder with psychosis
*****************
['ASD', 'FASD']
fasd
*****************
['ASD', 'FASD']
fasd
*****************
['Anxiety', 'Trauma']
anxiety with features of ptsd
*****************
['Depression', 'Disruptive, Impulse-Control, and Conduct']
with disruption in mood and conduct
*****************
['ASD', 'FASD']
fasd
*****************
['Neurodevelopmental', 'Intellectual Disability/Global Developmental Delay']
neurodevelopmental delay with intellectual disability
*****************
['Intellectual Disability/Global Developmental Delay', 'Personality']
borderline intellectual disability
*****************
['ASD', 'FASD']
fasd
*****************
['Schizophrenia Spectrum and Other Psychotic Disorders', 'Trauma']
significant psychosocial stressors related to lack of

*****************
['Depression', 'Trauma', 'Parent-Child Relational Issues']
adjustment disorder with disturbance of mood and behaviour secondary to conflict between parents
*****************
['Disruptive, Impulse-Control, and Conduct', 'Personality']
past conduct disorder with ongoing irritability as a personality trait
*****************
['Schizophrenia Spectrum and Other Psychotic Disorders', 'Depression']
cycling mood disorder with psychotic features
*****************
['Schizophrenia Spectrum and Other Psychotic Disorders', 'Depression']
unspecified depressive disorder with psychotic features
*****************
['Neurodevelopmental', 'ASD', 'FASD']
neurodevelopmental disability - fasd
*****************
['Trauma', 'Parent-Child Relational Issues']
 insecure attachment style with perceived threats to security of placement with grandparents
*****************
['Depression', 'Disruptive, Impulse-Control, and Conduct']
disruptive mood dysregulation disorder
*****************
['Anxiety', 'S

In [12]:
## Module 4: Read Reason for Referral, break into substrings, fill in binaries
one_hot_encode_reasons_for_referral(sheet1, IGNORE_KEYWORDS, REASONS_FOR_REFERRAL_KEYWORDS)

Referral reason: Violent outburst at school. Stabbed teacher with scissors in the hand. Had strangled staff, given them concussions, and more police men required to restrain him in the past 5 days.
Chief complaint: Very violent outburst at school - stabbed teacher in the hand, with scissors
**************************
Referral reason: Mania
Chief complaint: Mania
**************************
Referral reason: Decompensation, paranoia. 
Chief complaint: Decompensation and Paranoia
**************************
Referral reason: Ingestion of prescribed medication and stockpiling them for a week.
Chief complaint: Collective intake of Lamotragine, Concerta, Walbutrin, Guanfacine in large dose with intent of self harm
**************************
Referral reason: OCD Symptoms, specifically intrusive thoughts causing deterioration. 
Chief complaint: OCD Symptoms & Intrusive thoughts
**************************
Referral reason: OCD symptoms and deterioration of functioning. 
Chief complaint: OCD Symptom

In [13]:
## Module 5: Read all four columns, break into substrings, fill in binaries
one_hot_encode_symptoms(sheet1, IGNORE_KEYWORDS, SYMPTOMS_KEYWORDS)

In [14]:
## Output file
workbook.save("output-{}.xlsx".format(getTime()))